In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from rapidfuzz import process, fuzz

import warnings
import pandas as pd
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))

# Download Data

In [2]:
#From open states: bill_sponsorship
sponsorships=pd.read_csv('US_119_bill_sponsorships.csv')

#Congressional list with bioguide ids and districts
meta_data=pd.read_csv('119th Congress.csv')

#From open states: vote_counts (by vote_id; raw counts of individual votes)
counts=pd.read_csv('US_119_vote_counts.csv')

#From open states: votes (vote identifier; type of vote, result of vote,d ate of vote, bill_id)
votes=pd.read_csv('US_119_votes.csv')

#From open states: bills (bill information)
bills=pd.read_csv('US_119_bills.csv')

#From open states: vote_people (individual votes)
ind_votes=pd.read_csv('US_119_vote_people.csv')

In [3]:
meta_data.head()

,Name,Chamber,bioguide_id,State,District,Party
0,Barry Moore,House,M001212,Alabama,1,Republican
1,Shomari Figures,House,F000481,Alabama,2,Democratic
2,Mike D. Rogers,House,R000575,Alabama,3,Republican
3,Robert B. Aderholt,House,A000055,Alabama,4,Republican
4,Dale W. Strong,House,S001220,Alabama,5,Republican


# Match Names

The OpenStates data is not consistent on using bioguide_ids so this code was written to do a match on a shortened version of each congress person's name

In [4]:
#make a name_clean column for matching purposes
meta_data['Name'] = meta_data['Name'].str.replace(r'\b(jr|sr)\b\.?', '', case=False, regex=True)
sponsorships['name_clean'] = sponsorships['name'].str.lower().str.replace('.', '', regex=False).str.replace(',', '', regex=False)
meta_data['name_clean']=meta_data['Name'].str.lower().str.strip().str.replace('.', '', regex=False).str.replace(',', '', regex=False)

# Split the 'name_clean' column into three columns: first, middle, and last names.
def split_name(name):
    parts = name.split()
    if len(parts) == 1:
        return parts[0], '', ''
    elif len(parts) == 2:
        return parts[0], '', parts[1]
    elif len(parts) == 3:
        return parts[0], parts[1], parts[2]
    else:
        return parts[0], ' '.join(parts[1:-1]), parts[-1]


sponsorships[['first_name', 'middle_name', 'last_name']] = sponsorships['name_clean'].apply(lambda x: pd.Series(split_name(x)))
meta_data[['first_name', 'middle_name', 'last_name']] = meta_data['name_clean'].apply(lambda x: pd.Series(split_name(x)))

#limit first name to first 3 letters only
meta_data['first_name'] = meta_data['first_name'].str[:3]
sponsorships['first_name'] = sponsorships['first_name'].str[:3]

# make column that is short_name combining first_name and last_name columns
meta_data['short_name'] =  meta_data['last_name']+ ' ' + meta_data['first_name']
sponsorships['short_name'] = sponsorships['last_name']+ ' ' + sponsorships['first_name']


In [5]:
#function to perform fuzzy match on short_name columns
def fuzzy_merge_fast(df_1, df_2, key1, key2, threshold=70):
    choices = df_2[key2].dropna().unique().tolist()
    matches = df_1[key1].apply(
        lambda x: process.extractOne(x, choices, scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
    )
    df_1['matched_name'] = matches.apply(lambda x: x[0] if x else None)
    return df_1

#run fuzzy_match on short_name columns
sponsorships = fuzzy_merge_fast(sponsorships, meta_data, 'short_name', 'short_name', threshold=70)

sponsorships.head(2)

,id,name,entity_type,organization_id,person_id,bill_id,primary,classification,name_clean,first_name,middle_name,last_name,short_name,matched_name
0,cf11c552-461c-4c6e-b0ab-6ef7d6f43162,David G. Valadao,person,NaN,ocd-person/cf552a34-e9e1-5de9-9270-b5b441dd89fe,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,david g valadao,dav,g,valadao,valadao dav,valadao dav
1,b3d31feb-93a7-4882-95e5-0e24660571fc,John Garamendi,person,NaN,ocd-person/d4381c32-b267-5e35-b118-c0da3a0a80a1,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,john garamendi,joh,,garamendi,garamendi joh,garamendi joh


In [6]:
#find rows where there was a name match
matched = sponsorships[sponsorships['matched_name'].notna()].copy()

matched = matched.merge(
    meta_data[['short_name', 'bioguide_id', 'Party', 'Chamber', 'State', 'District']],
    left_on='matched_name',
    right_on='short_name',
    how='left'
)

matched.head(2)


,id,name,entity_type,organization_id,person_id,bill_id,primary,classification,name_clean,first_name,middle_name,last_name,short_name_x,matched_name,short_name_y,bioguide_id,Party,Chamber,State,District
0,cf11c552-461c-4c6e-b0ab-6ef7d6f43162,David G. Valadao,person,NaN,ocd-person/cf552a34-e9e1-5de9-9270-b5b441dd89fe,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,david g valadao,dav,g,valadao,valadao dav,valadao dav,valadao dav,V000129,Republican,House,California,22
1,b3d31feb-93a7-4882-95e5-0e24660571fc,John Garamendi,person,NaN,ocd-person/d4381c32-b267-5e35-b118-c0da3a0a80a1,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,john garamendi,joh,,garamendi,garamendi joh,garamendi joh,garamendi joh,G000559,Democratic,House,California,8


In [7]:
#find unmatched rows
unmatched = sponsorships[sponsorships['matched_name'].isna()].copy()
for col in ['bioguide_id', 'party', 'type', 'state', 'district']:
    unmatched[col] = None

In [8]:
#show unique names in unmatched dataframe
#(the results here should be delegates to congress which are not members and Marco Rubio -who left senate to be in cabinet and JD vance who is president of senate)
unmatched['name'].unique()


array(['James C. Moylan', 'Aumua Amata Coleman Radewagen',
       'Kimberlyn King-Hinds', 'Marco Rubio', 'J. D. Vance',
       'Jimmy Patronis'], dtype=object)

In [9]:
# delete unnecessary columns from matched

matched = matched.drop(columns=['name_clean', 'first_name', 'middle_name', 'last_name', 'short_name_x', 'matched_name', 'short_name_y'])


In [10]:
# Combine matched and unmatched rows
final_df = pd.concat([matched, unmatched], ignore_index=True)
# Save final result with required columns
output_cols = [
    'name', 'bill_id', 'primary', 'classification',
    'matched_name', 'bioguide_id', 'party', 'type', 'state', 'district'
]

final_df[output_cols].head()


,name,bill_id,primary,classification,matched_name,bioguide_id,party,type,state,district
0,David G. Valadao,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,NaN,V000129,NaN,NaN,NaN,NaN
1,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,NaN,G000559,NaN,NaN,NaN,NaN
2,Patrick Ryan,ocd-bill/65082e82-df48-420f-b139-443325ee91c6,True,primary,NaN,R000579,NaN,NaN,NaN,NaN
3,Jimmy Panetta,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,NaN,P000613,NaN,NaN,NaN,NaN
4,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,True,primary,NaN,H001090,NaN,NaN,NaN,NaN


# Create bill_sponsor dataframe

In [11]:
#Create dataframe of only primary sponsors of bills
primary_df = matched[matched['classification'] == 'primary'][['id','name','bill_id', 'bioguide_id', 'Party', 'Chamber']]
primary_df = primary_df.rename(columns={'Party': 'primary_party'})

#create dataframe of only cosponsors of bills
cosponsor_df = matched[matched['classification'] == 'cosponsor'][['name','bill_id', 'bioguide_id', 'Party', 'Chamber']]
cosponsor_df = cosponsor_df.rename(columns={'Party': 'cosponsor_party'})

In [12]:
#merge the primary and cosponsor dataframe
bill_sponsor = primary_df.merge(cosponsor_df, on='bill_id', how='left', suffixes=('_primary','_cosponsor'))

#add column to bill sponsor that shows True if bill has cross-party co-sponsor
bill_sponsor['Cross Party Sponsorship']=bill_sponsor['primary_party']!=bill_sponsor['cosponsor_party']




In [13]:
bill_sponsor.head()

,id,name_primary,bill_id,bioguide_id_primary,primary_party,Chamber_primary,name_cosponsor,bioguide_id_cosponsor,cosponsor_party,Chamber_cosponsor,Cross Party Sponsorship
0,8a396a28-95ee-4b6b-8b3a-30aeb3675121,Patrick Ryan,ocd-bill/65082e82-df48-420f-b139-443325ee91c6,R000579,Democratic,House,NaN,NaN,NaN,NaN,True
1,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,David G. Valadao,V000129,Republican,House,True
2,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,G000559,Democratic,House,False
3,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,Jimmy Panetta,P000613,Democratic,House,False
4,efdb1d22-8d0b-4ca8-9c49-28f325628ca5,Ryan K. Zinke,ocd-bill/3577abc0-2161-4f38-83be-77090867b3bd,Z000018,Republican,House,Troy Downing,D000634,Republican,House,False


In [14]:
# Replace results in Cross Party Sponsorship to be true if any co-sponsor is cross party

# Group by 'bill_id' and check if 'Cross Party Sponsorship' is True for any row within the group
cross_party_sponsorship_by_bill = bill_sponsor.groupby('bill_id')['Cross Party Sponsorship'].any()

# Update the 'Cross Party Sponsorship' column in the original DataFrame based on the grouped results
bill_sponsor['Cross Party Sponsorship'] = bill_sponsor['bill_id'].map(cross_party_sponsorship_by_bill)


In [15]:
bill_sponsor.head(3)

,id,name_primary,bill_id,bioguide_id_primary,primary_party,Chamber_primary,name_cosponsor,bioguide_id_cosponsor,cosponsor_party,Chamber_cosponsor,Cross Party Sponsorship
0,8a396a28-95ee-4b6b-8b3a-30aeb3675121,Patrick Ryan,ocd-bill/65082e82-df48-420f-b139-443325ee91c6,R000579,Democratic,House,NaN,NaN,NaN,NaN,True
1,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,David G. Valadao,V000129,Republican,House,True
2,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,G000559,Democratic,House,True


# Create Bill Vote Dataraframe

In [16]:
counts = counts.drop(columns=['id'])

In [17]:
# Add new columns to the votes DataFrame
votes['abstain'] = 0
votes['not voting'] = 0
votes['no'] = 0
votes['yes'] = 0

In [18]:
# Iterate through votes dataframe to make the vote results into columns instead of rows
for index, row in votes.iterrows():
    matching_counts = counts[counts['vote_event_id'] == row['id']]
    for _, count_row in matching_counts.iterrows():
        if 'abstain' in count_row['option']:
            votes.loc[index, 'abstain'] = count_row['value']
        if 'not voting' in count_row['option']:
            votes.loc[index, 'not voting'] = count_row['value']
        if 'no' in count_row['option']:
            votes.loc[index, 'no'] = count_row['value']
        if 'yes' in count_row['option']:
            votes.loc[index, 'yes'] = count_row['value']


In [19]:
# add total_votes column
votes['total_votes'] = votes['abstain'] + votes['not voting'] + votes['no'] + votes['yes']

In [20]:
votes.head(3)

,id,identifier,motion_text,motion_classification,start_date,result,organization_id,bill_id,bill_action_id,jurisdiction,session_identifier,abstain,not voting,no,yes,total_votes
0,ocd-vote/a31b4e67-7b3d-45b0-9bc7-3461a975d202,us-2025-lower-3,On Ordering the Previous Question,['passage'],2025-01-03T22:54:00+00:00,pass,ocd-organization/24af4233-d9b5-5933-91b2-51d29...,ocd-bill/2c089ed7-8815-4d47-b432-e93c7c9eb1df,NaN,United States,119,0,8,210,216,434
1,ocd-vote/f28036c3-7abf-4509-9b1f-b133b3e2e329,us-2025-lower-4,On Motion to Commit with Instructions,['passage'],2025-01-03T23:01:00+00:00,fail,ocd-organization/24af4233-d9b5-5933-91b2-51d29...,ocd-bill/2c089ed7-8815-4d47-b432-e93c7c9eb1df,NaN,United States,119,0,11,214,209,434
2,ocd-vote/86b3245c-b8fd-41dc-a080-e2c1a7f3a1a8,us-2025-lower-5,On Agreeing to the Resolution,['passage'],2025-01-03T23:08:00+00:00,pass,ocd-organization/24af4233-d9b5-5933-91b2-51d29...,ocd-bill/2c089ed7-8815-4d47-b432-e93c7c9eb1df,NaN,United States,119,0,10,209,215,434


In [21]:
# Merge votes and bills DataFrames
merged_votes_bills = votes.merge(bills, left_on='bill_id', right_on='id', how='left')

merged_votes_bills=merged_votes_bills[merged_votes_bills['classification']=="['bill']"]

In [22]:
# Keep only votes that are on passage in either house or senate

passage_votes = merged_votes_bills[merged_votes_bills['motion_text'].str.startswith('On Passage', na=False)]


In [23]:
passage_votes=passage_votes[['id_x', 'motion_text', 'motion_classification',
       'result', 'bill_id', 'abstain', 'not voting', 'no',
       'yes', 'total_votes', 'identifier_y', 'title', 'classification',
       'organization_classification']]

# Rename columns
passage_votes = passage_votes.rename(columns={'id_x': 'vote_id'})
passage_votes = passage_votes.rename(columns={'identifier_y': 'Bill Number'})

In [24]:
# prompt: merge passage_votes and bill_sponsor dataframes on bill_id

# Merge passage_votes and bill_sponsor DataFrames
passage_votes = passage_votes.merge(bill_sponsor, on='bill_id', how='left')

passage_votes.head(2)


,vote_id,motion_text,motion_classification,result,bill_id,abstain,not voting,no,yes,total_votes,...,id,name_primary,bioguide_id_primary,primary_party,Chamber_primary,name_cosponsor,bioguide_id_cosponsor,cosponsor_party,Chamber_cosponsor,Cross Party Sponsorship
0,ocd-vote/9ca8ab8f-f167-439b-9ce9-58c940210c6d,On Passage,['passage'],pass,ocd-bill/bf8ba42d-25e2-4f17-8a43-efd0018a4138,0,11,159,264,434,...,ca35399c-5135-4002-aa4b-3d3707e8f384,Mike Collins,C001129,Republican,House,Rick W. Allen,A000372,Republican,House,True
1,ocd-vote/9ca8ab8f-f167-439b-9ce9-58c940210c6d,On Passage,['passage'],pass,ocd-bill/bf8ba42d-25e2-4f17-8a43-efd0018a4138,0,11,159,264,434,...,ca35399c-5135-4002-aa4b-3d3707e8f384,Mike Collins,C001129,Republican,House,Marjorie Taylor Greene,G000596,Republican,House,True


In [25]:
passage_votes[passage_votes['primary_party']=="Democratic"]

,vote_id,motion_text,motion_classification,result,bill_id,abstain,not voting,no,yes,total_votes,...,id,name_primary,bioguide_id_primary,primary_party,Chamber_primary,name_cosponsor,bioguide_id_cosponsor,cosponsor_party,Chamber_cosponsor,Cross Party Sponsorship


In [26]:
# prompt: sum the total_votes column in passage_votes

total_votes_sum = passage_votes['total_votes'].sum()
print(total_votes_sum)


373414


In [27]:
# make list of vote_ids of passage votes
passage_votes_ids=passage_votes['vote_id'].unique()

In [28]:
#  change column name in ind_votes from note to voter_bioguide_id
ind_votes = ind_votes.rename(columns={'note': 'voter_bioguide_id'})


In [29]:
# Filter ind_votes to keep only rows where 'vote_event_id' is in passage_votes_ids
ind_votes = ind_votes[ind_votes['vote_event_id'].isin(passage_votes_ids)]

In [30]:
#this should be the same as the total_votes_sum above
len(ind_votes)

7659

In [31]:
ind_votes.head(3)

,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id
1265,36656e82-50a6-428d-97da-035257cd72f9,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Rose,ocd-person/a30feb1f-c9fe-5612-808e-8fe5a55b76dc,R000612
1423,8d362275-8a0f-4bc3-828e-4eea8d0d5042,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,no,Ross,ocd-person/ea21df34-9ef4-5746-b83a-700bdc2d1918,R000305
1424,51a77ce4-1f30-403f-aac0-bd5a7fd0fd3d,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Rouzer,ocd-person/5e9726e3-2aae-5708-9227-0e95ac43c203,R000603


In [32]:
#create dataframe of house only votes (house members have
house_ind_votes=ind_votes[ind_votes['voter_bioguide_id'].str.len() >= 5]

In [33]:
# Filter ind_votes to keep rows where the length of the voter_bioguide_id column is less than 5.
sen_ind_votes=ind_votes[ind_votes['voter_bioguide_id'].str.len() < 5]

# Split the 'voter_name' column into two columns: 'Last Name' and 'State'.
# Escape the opening parenthesis in the split pattern to treat it as a literal character.
sen_ind_votes[['Last Name', 'State']] = sen_ind_votes['voter_name'].str.split(r"(", n=1, expand=True)

sen_ind_votes['Last Name']=sen_ind_votes['Last Name'].str.lower().str.strip()
sen_ind_votes['State'] = sen_ind_votes['State'].str[-3:-1]
sen_ind_votes.head(3)



,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id,Last Name,State
15327,e7d684c4-bcbc-4827-bf65-19ff98296dbe,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Alsobrooks (D-MD),NaN,S428,alsobrooks,MD
15328,880e08ff-de48-499f-933c-b5141e2e5974,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Baldwin (D-WI),NaN,S354,baldwin,WI
15329,92888a5b-34a5-4c84-a6f4-60fee8701299,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,yes,Banks (R-IN),NaN,S429,banks,IN


In [34]:
# Iterate through the 'Last Name' column and modify strings with more than one word
for index, row in sen_ind_votes.iterrows():
    last_name = row['Last Name']
    if ' ' in last_name:
        new_last_name = last_name.split(' ', 1)[1]  # Split at the first space and take the second part
        sen_ind_votes.loc[index, 'Last Name'] = new_last_name


In [35]:
senate_meta_data=meta_data[meta_data['Chamber']=='Senate']
senate_meta_data.head(3)

,Name,Chamber,bioguide_id,State,District,Party,name_clean,first_name,middle_name,last_name,short_name
7,Katie Boyd Britt,Senate,B001319,Alabama,NaN,Republican,katie boyd britt,kat,boyd,britt,britt kat
8,Tommy Tuberville,Senate,T000278,Alabama,NaN,Republican,tommy tuberville,tom,,tuberville,tuberville tom
10,Lisa Murkowski,Senate,M001153,Alaska,NaN,Republican,lisa murkowski,lis,,murkowski,murkowski lis


In [36]:
# Look up senators by last name to get correct bioguide_ids in ind_votes file

# Create an empty list to store the results
bioguide_ids = []

# Iterate through the 'voter_id' column in sen_ind_votes
for last_name in sen_ind_votes['Last Name']:
    # Look up the last name in the senate_meta_data DataFrame
    match = senate_meta_data[senate_meta_data['last_name'] == last_name]

    # If a match is found, append the corresponding bioguide_id to the list
    if not match.empty:
        bioguide_ids.append(match['bioguide_id'].iloc[0])
    else:
        bioguide_ids.append(None)  # Append None if no match is found

# Add the bioguide_ids as a new column to sen_ind_votes
sen_ind_votes['bioguide_id'] = bioguide_ids


In [37]:
sen_ind_votes.head(5)

,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id,Last Name,State,bioguide_id
15327,e7d684c4-bcbc-4827-bf65-19ff98296dbe,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Alsobrooks (D-MD),NaN,S428,alsobrooks,MD,A000382
15328,880e08ff-de48-499f-933c-b5141e2e5974,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Baldwin (D-WI),NaN,S354,baldwin,WI,B001230
15329,92888a5b-34a5-4c84-a6f4-60fee8701299,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,yes,Banks (R-IN),NaN,S429,banks,IN,B001299
15330,f862b085-ab6c-4d28-966d-8afac3a04bc5,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,yes,Barrasso (R-WY),NaN,S317,barrasso,WY,B001261
15331,10d184f0-1ddb-4b68-bf50-3824388265d8,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Bennet (D-CO),NaN,S330,bennet,CO,B001267


In [38]:
# Delete the 'voter_bioguide_id' column from sen_ind_votes
sen_ind_votes = sen_ind_votes.drop(columns=['voter_bioguide_id', 'Last Name', 'State'])

# Rename the 'bioguide_id' column to 'voter_bioguide_id' in the 'meta_data' DataFrame
sen_ind_votes = sen_ind_votes.rename(columns={'bioguide_id': 'voter_bioguide_id'})



In [39]:
sen_ind_votes.head()

,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id
15327,e7d684c4-bcbc-4827-bf65-19ff98296dbe,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Alsobrooks (D-MD),NaN,A000382
15328,880e08ff-de48-499f-933c-b5141e2e5974,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Baldwin (D-WI),NaN,B001230
15329,92888a5b-34a5-4c84-a6f4-60fee8701299,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,yes,Banks (R-IN),NaN,B001299
15330,f862b085-ab6c-4d28-966d-8afac3a04bc5,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,yes,Barrasso (R-WY),NaN,B001261
15331,10d184f0-1ddb-4b68-bf50-3824388265d8,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Bennet (D-CO),NaN,B001267


In [40]:
house_ind_votes.head(2)

,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id
1265,36656e82-50a6-428d-97da-035257cd72f9,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Rose,ocd-person/a30feb1f-c9fe-5612-808e-8fe5a55b76dc,R000612
1423,8d362275-8a0f-4bc3-828e-4eea8d0d5042,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,no,Ross,ocd-person/ea21df34-9ef4-5746-b83a-700bdc2d1918,R000305


In [41]:
sen_ind_votes_blank=sen_ind_votes[sen_ind_votes['voter_bioguide_id'].isna()]

#run blanks - should only be Marco Rubio
sen_ind_votes_blank

,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id
15402,cdcfd815-c445-43d5-b1a7-a0d170e67c9c,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,yes,Rubio (R-FL),NaN,None


In [42]:
#recombine house and senate data
ind_votes=pd.concat([sen_ind_votes,house_ind_votes])

In [43]:
# Merge ind_votes with meta_data DataFrame
ind_votes = ind_votes.merge(meta_data, left_on='voter_bioguide_id', right_on='bioguide_id', how='left')


In [44]:
ind_votes.head(2)

,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id,Name,Chamber,bioguide_id,State,District,Party,name_clean,first_name,middle_name,last_name,short_name
0,e7d684c4-bcbc-4827-bf65-19ff98296dbe,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Alsobrooks (D-MD),NaN,A000382,Angela D. Alsobrooks,Senate,A000382,Maryland,NaN,Democratic,angela d alsobrooks,ang,d,alsobrooks,alsobrooks ang
1,880e08ff-de48-499f-933c-b5141e2e5974,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,Baldwin (D-WI),NaN,B001230,Tammy Baldwin,Senate,B001230,Wisconsin,NaN,Democratic,tammy baldwin,tam,,baldwin,baldwin tam


In [45]:
#drop voter_name, voter_bioguide_id, state, district columns from ind_votes df
ind_votes = ind_votes.drop(columns=['voter_name', 'voter_bioguide_id', 'State', 'District', 'name_clean','first_name','middle_name','last_name','short_name'])


In [46]:
ind_votes.head(2)

,id,vote_event_id,option,voter_id,Name,Chamber,bioguide_id,Party
0,e7d684c4-bcbc-4827-bf65-19ff98296dbe,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,NaN,Angela D. Alsobrooks,Senate,A000382,Democratic
1,880e08ff-de48-499f-933c-b5141e2e5974,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,NaN,Tammy Baldwin,Senate,B001230,Democratic


In [47]:
# iterate over ind_votes df, matching on ind_votes['vote_event_id'] and passage_votes['vote_id'], adding a column to ind_votes with passage_votes['primary_party']

# Add a new column to ind_votes to store the primary party
ind_votes['primary_party'] = None

# Iterate over the ind_votes DataFrame
for index, row in ind_votes.iterrows():
  # Find the matching row in passage_votes based on vote_event_id
  matching_rows = passage_votes[passage_votes['vote_id'] == row['vote_event_id']]
  # If a match is found, update the primary_party column in ind_votes
  if not matching_rows.empty:
    ind_votes.loc[index, 'primary_party'] = matching_rows['primary_party'].iloc[0]
    ind_votes.loc[index, 'Cross Party Sponsorship'] = matching_rows['Cross Party Sponsorship'].iloc[0]


In [48]:
# Rename columns to clarify parties
ind_votes = ind_votes.rename(columns={'primary_party': 'Bill_Sponsor_Party'})
ind_votes = ind_votes.rename(columns={'Party': 'Voter_Party'})

In [49]:
ind_votes.head(2)

,id,vote_event_id,option,voter_id,Name,Chamber,bioguide_id,Voter_Party,Bill_Sponsor_Party,Cross Party Sponsorship
0,e7d684c4-bcbc-4827-bf65-19ff98296dbe,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,NaN,Angela D. Alsobrooks,Senate,A000382,Democratic,Republican,True
1,880e08ff-de48-499f-933c-b5141e2e5974,ocd-vote/9dffa967-bec7-4bd0-9522-32a79e9286e3,no,NaN,Tammy Baldwin,Senate,B001230,Democratic,Republican,True


In [50]:
#add boolean columns to determine if specific vote meets critera

ind_votes['Vote for Other Party']=((ind_votes['option']=='yes')&(ind_votes['Bill_Sponsor_Party']!=ind_votes['Voter_Party']))

ind_votes['Vote for Cosponsored Bill']=((ind_votes['option']=='yes')&(ind_votes['Cross Party Sponsorship']==True))

# Calculate the sum of those categories
sum_vote_for_other_party = ind_votes['Vote for Other Party'].sum()

sum_for_cosponsored = ind_votes['Vote for Cosponsored Bill'].sum()

#Print sums
print('Votes for other party:',sum_vote_for_other_party)

print('Vote for Cosponsored Bill',sum_for_cosponsored)

Votes for other party: 810
Vote for Cosponsored Bill 2201


In [51]:
meta_data.head(3)

,Name,Chamber,bioguide_id,State,District,Party,name_clean,first_name,middle_name,last_name,short_name
0,Barry Moore,House,M001212,Alabama,1,Republican,barry moore,bar,,moore,moore bar
1,Shomari Figures,House,F000481,Alabama,2,Democratic,shomari figures,sho,,figures,figures sho
2,Mike D. Rogers,House,R000575,Alabama,3,Republican,mike d rogers,mik,d,rogers,rogers mik


In [52]:
#Keep only the rows that have a True in one of the relevant columns
cross_votes=ind_votes[(ind_votes['Vote for Other Party'] == True) |
                           (ind_votes['Vote for Cosponsored Bill'] == True)]

In [53]:
#create a summary dataframe with only name, bioguide_id and vote counts

cross_votes_summary = cross_votes.groupby('Name').agg(
    {'Vote for Other Party': 'sum', 'Vote for Cosponsored Bill':'sum'}
).reset_index()

# Merge with meta_data dataframe to get bioguide_id
cross_votes_summary = cross_votes_summary.merge(meta_data[['Name', 'bioguide_id']], on='Name', how='left')

# Reorder columns
cross_votes_summary = cross_votes_summary[['Name', 'bioguide_id','Vote for Other Party', 'Vote for Cosponsored Bill']]

cross_votes_summary.head()


,Name,bioguide_id,Vote for Other Party,Vote for Cosponsored Bill
0,Aaron Bean,B001314,0,7
1,Abraham Hamadeh,H001098,0,7
2,Adam Gray,G000605,11,6
3,Adam Smith,S000510,1,1
4,Addison McDowell,M001240,0,7


In [54]:
# Merge cross_votes_summary with ind_votes on 'bioguide_id'
meta_data = meta_data.merge(cross_votes_summary, on='bioguide_id', how='left')
meta_data.head()

,Name_x,Chamber,bioguide_id,State,District,Party,name_clean,first_name,middle_name,last_name,short_name,Name_y,Vote for Other Party,Vote for Cosponsored Bill
0,Barry Moore,House,M001212,Alabama,1,Republican,barry moore,bar,,moore,moore bar,Barry Moore,0.0,7.0
1,Shomari Figures,House,F000481,Alabama,2,Democratic,shomari figures,sho,,figures,figures sho,Shomari Figures,6.0,4.0
2,Mike D. Rogers,House,R000575,Alabama,3,Republican,mike d rogers,mik,d,rogers,rogers mik,Mike D. Rogers,0.0,7.0
3,Robert B. Aderholt,House,A000055,Alabama,4,Republican,robert b aderholt,rob,b,aderholt,aderholt rob,Robert B. Aderholt,0.0,7.0
4,Dale W. Strong,House,S001220,Alabama,5,Republican,dale w strong,dal,w,strong,strong dal,Dale W. Strong,0.0,7.0


In [55]:
# prompt: drop name_y column from meta_data df and change Name_X column to Name

meta_data = meta_data.drop(columns=['Name_y','name_clean','first_name','middle_name','last_name','short_name'])
meta_data = meta_data.rename(columns={'Name_x': 'Name'})


In [56]:
meta_data.head()

,Name,Chamber,bioguide_id,State,District,Party,Vote for Other Party,Vote for Cosponsored Bill
0,Barry Moore,House,M001212,Alabama,1,Republican,0.0,7.0
1,Shomari Figures,House,F000481,Alabama,2,Democratic,6.0,4.0
2,Mike D. Rogers,House,R000575,Alabama,3,Republican,0.0,7.0
3,Robert B. Aderholt,House,A000055,Alabama,4,Republican,0.0,7.0
4,Dale W. Strong,House,S001220,Alabama,5,Republican,0.0,7.0


In [57]:
# Fill NaN values in 'Vote for Other Party' and 'Vote Against Own Party' with 0
meta_data.fillna({'Vote for Other Party': 0}, inplace=True)
meta_data.fillna({'Vote for Cosponsored Bill': 0}, inplace=True)

# Display the updated ind_votes DataFrame
meta_data.head(2)

,Name,Chamber,bioguide_id,State,District,Party,Vote for Other Party,Vote for Cosponsored Bill
0,Barry Moore,House,M001212,Alabama,1,Republican,0.0,7.0
1,Shomari Figures,House,F000481,Alabama,2,Democratic,6.0,4.0


In [58]:
len(meta_data)

536

In [59]:
meta_data_house_rep=meta_data[(meta_data['Chamber']=="House") & (meta_data['Party']=='Republican')]
meta_data_house_dem=meta_data[(meta_data['Chamber']=="House") & (meta_data['Party']=='Democratic')]
meta_data_senate_rep=meta_data[(meta_data['Chamber']=="Senate") & (meta_data['Party']=='Republican')]
meta_data_senate_dem=meta_data[(meta_data['Chamber']=="Senate") & ((meta_data['Party']=='Democratic') |(meta_data['Party']=='Independent') )]

In [60]:
len(meta_data_house_rep)+len(meta_data_house_dem)+len(meta_data_senate_rep)+len(meta_data_senate_dem)

536

In [61]:
#G cosponsored
#H Sponsored by other party

In [62]:
def party_mean(df):
    mean_other_vote=round(df['Vote for Other Party'].mean(),2)
    std_other_vote=round(df['Vote for Other Party'].std(),2)
    df['norm_H']=norm.cdf(df['Vote for Other Party'],mean_other_vote,std_other_vote)*100
    mean_cosp_vote=round(df['Vote for Cosponsored Bill'].mean(),2)
    std_other_vote=round(df['Vote for Cosponsored Bill'].std(),2)
    df['norm_G']=norm.cdf(df['Vote for Cosponsored Bill'],mean_other_vote,std_other_vote)*100
    return(df)
    

In [64]:
meta_data.to_csv('votes.csv', index=False)